# Automation tools for searching the parameter space

In this tutorial we will cover tools that implement the following methods for searching the parameter space and finding the best model in SensiML cloud.

1. Feature Explosion
2. Feature Selection
3. Grid Search
4. Survival Search


In [1]:
import pandas as pd

from sensiml import SensiML
from sensiml.widgets import QueryWidget, AutoSenseWidget, DownloadWidget

dsk = SensiML()
dsk.project ='Parameter Optimization Tutorial'
dsk.pipeline = 'Easy_Pipeline_Button'

In [4]:
df = pd.read_csv('Support/grid_search_tutorial_activity_data.csv')
sensor_columns = ['AccelerometerX', 'AccelerometerY', 'AccelerometerZ']

dsk.upload_dataframe('grid_dataframe', df)

df.head()

Uploading file "grid_dataframe" to KB Cloud.
Upload of file "grid_dataframe.csv"  to KB Cloud completed.


,Subject,Class,AccelerometerX,AccelerometerY,AccelerometerZ
0,U001,0,-317,-3000,925
1,U001,0,-284,-2968,903
2,U001,0,-243,-2987,933
3,U001,0,-193,-3051,936
4,U001,0,-150,-3059,915


In [5]:
dsk.pipeline.reset()
dsk.pipeline.set_input_data('grid_dataframe.csv',  data_columns=sensor_columns,
                                                  group_columns=['Class','Subject'],
                                                  label_column='Class',
                                                  )

dsk.pipeline.add_transform('Windowing')

dsk.pipeline.add_transform('MSE Filter', params={'input_column':sensor_columns[0]})

## 1. Feature Explosion

The feature generation step is key here, notice how we are using subtype calls. Subtype calls encompass large groups of feature generators. Feature explosion is a powerful technique where we generate massive amounts of features without making any guesses about which will be the best suited to the classification task. The downside to feature explosion is the danger of overfitting. To avoid that we implement techniques such as feature selectors as well as cross validation in our model building.

In [6]:
# Feature Generation
dsk.pipeline.add_feature_generator([{'subtype_call':'Time', 'params':{'sample_rate':100}},
                                    {'subtype_call':'Rate of Change'},
                                    {'subtype_call':'Statistical'},
                                    {'subtype_call':'Energy'},
                                    {'subtype_call':'Amplitude', 'params':{'smoothing_factor':9}}
                                    ],
                                    function_defaults={'columns':sensor_columns},
                                    )


# Scale to 8 bit representation for classification 
dsk.pipeline.add_transform('Min Max Scale')

## 2. Feature Selection <a id='Feature_selection_intro'></a>

Now that features have been generated in the pipeline and we have a large set of candidate features,  we need to select which of those are the best at discriminating between our labels or "y-values". That's where the selection process comes in to play. 

Imagine running through the dozens of features you've generated on hardware. You may not get as accurate results, and the operation may take a long time to run. Selecting the best features from that set will increase accuracy as well as vastly increase efficiency on hardware.

Much like the generators, selectors are function calls that are then placed into a <b>selector call set</b>

#### Selector Calls <a id='selectorcalls'></a>

Much like the generator calls before them, selector calls can be created by retrieving existing functions from the server. We can then populate their expected inputs, preparing them to go into a selector call set. Now we can add them to the set. Order matters in this, as the output of one selector automatically feeds into another. Also note that Recursive Feature Elimination can take a long time to run, so its best to remove some features using some faster feature selectors first. 

In [10]:
dsk.pipeline.add_feature_selector([
                                {"name": "Correlation Threshold","params":{'threshold':0.85}},
                                {"name": "Variance Threshold", "params":{'threshold':0.05}},
                                {"name":"Recursive Feature Elimination","params":{"method":"Log R", "number_of_features":20 }}],
                                  params = {"number_of_features":20,})

In [11]:
dsk.pipeline.set_validation_method('Stratified K-Fold Cross-Validation', params={'number_of_folds':3})

dsk.pipeline.set_classifier('PME', params={"classification_mode":'RBF','distance_mode':'L1'})

dsk.pipeline.set_training_algorithm('Hierarchical Clustering with Neuron Optimization', params = {'number_of_neurons':10})

dsk.pipeline.set_tvo({'validation_seed':0})

In [12]:
dsk.pipeline.describe()

------------------------------------------------------------------------
 0.     Name: grid_dataframe.csv        		Type: featurefile              
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
	Param: window_size: 250
	Param: delta: 250
	Param: train_delta: 0
	Param: return_segment_index: False
------------------------------------------------------------------------
 2.     Name: MSE Filter                		Type: transform                
------------------------------------------------------------------------
	Param: MSE_target: -1.0
	Param: MSE_threshold: 0.01
------------------------------------------------------------------------
 3.     Name: generator_set             		Type: generatorset             
-------------------------------------

## 3. Grid Search Optimization

In order to optimize the model performance often requires searching over a large parameter space. A common method of performing this search is grid search. In this tutorial we will demonstrate how to use grid search in knowledge builder to aid in building better optimized models. On the server side we take advantage of the parallelizable nature of the pipelines as well as optimizations for training algorithms to speed up the computation. This makes it possible to search large parameter spaces quickly and efficiently. After performing the grid search we rank each pipeline based on the f1 score, precision and sensitivity so that you can choose the best performing combination to build a knowledge pack with.


#### Grid Search Syntax

Now that we have a pipeline that works, we would like to search the parameter space to further optimize the model. To do this we will call the sandboxes grid search function "sb.grid_search()" and pass in a list of grid_params to search over.

Grid params is a nested python dictionary object. 

    grid_params = {"Name Of Function":{"Name of Parameter":[ A, B, C]}} 

Where A, B and C are the parameters to search over. Additionally, for each step you may want to search over more than one of a functions configurable parameters. To do this simply add another element to the functions dictionary.

    grid_params = {"Name Of Function":{"Name of Parameter 1":[ A, B, C],
                                       "Name of Parameter 2":[ D, E]}}
                                   
This will tell grid search to search over 6 different parameter spaces. 

You can also specify more than one step to search over in grid params. This is done through simply adding another element to the Function level of the grid_params dictionary.

    grid_params = {"Name Of Function":{"Name of Parameter 1":[ A, B, C],
                                       "Name of Parameter 2":[ D, E]},
                   "Name of Function 2":{"Name of Paramter":[1, 2, 3, 4, 5, 6]}}
                   
For the TVO step we currently only allow modification of the parameters of the training algorithm. To access the grid parameter space, use the name of the training algorithm as shown in the example below.

In [13]:
grid_params = {'Windowing':{"window_size": [100,200],'delta':[100]},
              'selector_set': {"Recursive Feature Elimination":{'number_of_features':[10, 20]}},
              'Hierarchical Clustering with Neuron Optimization': {'number_of_neurons':[10,20]}
              }

results, stats = dsk.pipeline.grid_search(grid_params)

Executing Pipeline with Steps:

------------------------------------------------------------------------
 0.     Name: grid_dataframe.csv        		Type: featurefile              
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
------------------------------------------------------------------------
 2.     Name: MSE Filter                		Type: transform                
------------------------------------------------------------------------
------------------------------------------------------------------------
 3.     Name: generator_set             		Type: generatorset             
------------------------------------------------------------------------
------------------------------------------------------------------------
 4.     Name: M

#### f1, precision and sensitivity score for each grid point

The output from grid search is a dataframe containing the f1, precision and sensitivity scores from each permutation of the pipeline. For cross-fold validation these are the average over all models. Below we show the Pandas dataframe functions for sorting by multiple columns in either ascending/descending order.

In [14]:
results.sort_values(['f1_score','training_method.number_of_neurons'], ascending=[False, True]).head()

,f1_score,window_size,training_method.number_of_neurons,sensitivity_std,sensitivity,delta,precision,Recursive Feature Elimination.number_of_features,precision_std,f1_score_std
0,97.529103,100,10,3.593867,95.737795,100,99.659864,10,0.481025,2.241235
1,97.021414,100,10,4.565224,95.116188,100,99.322695,20,0.620942,2.831575
2,96.317736,200,10,4.207666,93.637990,100,99.645390,20,0.501494,2.623715
3,94.634360,200,10,4.778882,90.758436,100,99.645390,10,0.501494,2.504950
4,92.414888,100,20,5.730792,87.913311,100,99.346405,20,0.924323,4.416869


## 4. Optimizating Parameters Using the Automation Genetic Algorithm

Another way to find optimal parameters is with a genetic algorithm. Instead of searching a large parameter space exhaustively to find the single best combination of parameters, the genetic algorithm starts with a small randomized population of parameter combinations, generates models from them and tests them, keeps a subset of high-performing combinations, and then recombines those "survivors" in different ways (see: crossover and mutation) and repeats the process over again. The offspring of good parameter combinations are usually also good and sometimes are significantly better than their parents. As the algorithm repeats each successive generation, it often finds a near-optimal model without trying as many configurations as grid search.

In this tutorial we will demonstrate how to use the Auto command to apply the genetic algorithm to your custom pipeline. On the server side, the pipelines are run in parallel and results are ranked by a fitness score which takes into account the model's F1 score, precision, sensitivity, and other metrics. You can learn more about the these performance metrics and the many automation options in KB Basics Tutorial 7.

#### Set the Auto Command with the Custom seed
To automate parameter selection on this pipeline, generate an Auto call with Snippets. Select the "Custom" seed to tell the server to use your defined pipeline instead of a preset template. 

    dsk.snippets.Auto.Custom()      
        
Note: This only generates the code, but doesn't execute it. Execute the cell a second time to start the automated pipeline.

In [19]:
results, summary = dsk.pipeline.auto({'seed': 'Custom', 'params':{"search_steps": ['selectorset', 'tvo'], 
								"population_size": 10, 
								"iterations": 1, 
								"mutation_rate": 0.1, 
								"recreation_rate": 0.1, 
								"survivor_rate": 0.5, 
								"reset": True, 
								"run_parallel": True, 
								"validation_method": 'K-Fold', 
								"max_minutes": 10, 
								"fitness": {'f1_score': 0.0, 'features': 0.3, 'sensitivity': 0.7, 'precision': 0.0, 'neurons': 0.5, 'positive_predictive_rate': 0.0, 'specificity': 0.0, 'accuracy': 1.0}}})

Running Auto Pipeline with Custom Seed:

------------------------------------------------------------------------
 0.     Name: grid_dataframe.csv        		Type: featurefile              
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
	Param: window_size: 250
	Param: delta: 250
	Param: train_delta: 0
	Param: return_segment_index: False
------------------------------------------------------------------------
 2.     Name: MSE Filter                		Type: transform                
------------------------------------------------------------------------
	Param: MSE_target: -1.0
	Param: MSE_threshold: 0.01
------------------------------------------------------------------------
 3.     Name: generator_set             		Type: generatorset          

#### Inspect the Fitness Results

The fitness summary contains everything you need to evaluate the best models found by the algorithm. By default, the first run only does one iteration, so the models may not be very impressive. Take a look at the KB Basics tutorial about automation for more information about the fitness summary and all of its metrics.

In [20]:
summary['fitness_summary']

,f1_score,pipeline,features,sensitivity,specificity,iteration,precision,neurons,positive_predictive_rate,fitness,best_model,knowledgepack,accuracy
0,96.718051,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",11.0,94.277778,0.0,0,100.000000,5.0,100.000000,2.371426,Fold 4,30505062-fe4b-40ca-b516-14234cc22498,95.715121
1,96.718051,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",14.0,94.277778,0.0,0,100.000000,5.0,100.000000,2.364340,Fold 4,e678f51b-92b9-41e9-95d6-370c08be5790,95.715121
2,78.018231,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",31.0,74.041958,0.0,0,87.980159,19.6,87.980159,1.891726,Fold 2,ad714869-5a11-4fb8-8173-4976761ccfce,72.382623
3,76.640054,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",21.0,67.648213,0.0,0,94.482295,23.8,94.482295,1.828351,Fold 0,cf0a9868-c2c8-485c-b479-3feac62d8a05,69.812030
4,59.822189,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",12.0,44.137529,0.0,0,100.000000,5.0,100.000000,1.515067,Fold 3,ce277eef-c41e-4cdc-800d-215468093825,45.413534
5,61.991864,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",52.0,51.217884,0.0,0,95.903846,20.6,95.903846,1.451389,Fold 3,NaN,49.680033
6,58.870911,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",43.0,43.349650,0.0,0,100.000000,5.0,100.000000,1.424602,Fold 1,NaN,44.241437
7,58.822714,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",62.0,47.032958,0.0,0,97.032051,20.8,97.032051,1.390617,Fold 3,NaN,48.973266
8,56.747806,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",17.0,42.227661,0.0,0,100.000000,14.0,100.000000,1.387093,Fold 4,NaN,38.677527
9,53.391290,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",17.0,38.879176,0.0,0,100.000000,19.0,100.000000,1.296734,Fold 3,NaN,33.954052


#### Additional Iterations
To let the algorithm do a few more iterations, set 'iterations' equal to 2 and set the 'reset' option to False (this tells the server you do NOT want to re-initialize).

In [21]:
results, summary = dsk.pipeline.auto({'seed': 'Custom', 
                                      'params': {'iterations': 2, 
                                                 'reset': False}})

Running Auto Pipeline with Custom Seed:

------------------------------------------------------------------------
 0.     Name: grid_dataframe.csv        		Type: featurefile              
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
	Param: window_size: 250
	Param: delta: 250
	Param: train_delta: 0
	Param: return_segment_index: False
------------------------------------------------------------------------
 2.     Name: MSE Filter                		Type: transform                
------------------------------------------------------------------------
	Param: MSE_target: -1.0
	Param: MSE_threshold: 0.01
------------------------------------------------------------------------
 3.     Name: generator_set             		Type: generatorset          

In [35]:
 summary['fitness_summary'].head()

,f1_score,pipeline,features,sensitivity,fitness,iteration,precision,neurons,positive_predictive_rate,specificity,best_model,knowledgepack,accuracy
0,96.718051,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",11.0,94.277778,2.371426,0,100.000000,5.0,100.000000,0.0,Fold 4,52f0bb9d-f390-4b4e-830d-d363e08ccc61,95.715121
1,96.718051,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",11.0,94.277778,2.371426,1,100.000000,5.0,100.000000,0.0,Fold 4,045211ae-d2fe-4456-9b31-ab40bf672e3e,95.715121
2,96.718051,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",14.0,94.277778,2.364340,0,100.000000,5.0,100.000000,0.0,Fold 4,dd5020c9-33f4-496f-a5ac-f9590c97d96d,95.715121
3,79.304488,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",21.0,76.373543,1.986805,1,89.191822,18.2,89.191822,0.0,Fold 1,4da64094-b2aa-43e7-8d04-06901c232d84,77.345029
4,78.018231,"[{""name"": ""grid_dataframe.csv"", ""outputs"": [""t...",31.0,74.041958,1.891726,0,87.980159,19.6,87.980159,0.0,Fold 2,884eff41-1839-442a-9888-d6d92960213b,72.382623


#### Look at the Optimal Features and Parameters
When you think you have found an interesting model, you can request the knowledgepack object from its ID in the summary table and view its features and pipeline, containing the optimized parameters.

In [32]:
kp_uuid = summary['fitness_summary'].iloc[0]['knowledgepack']

kp = dsk.get_knowledgepack(kp_uuid)

pd.DataFrame(kp.feature_summary)

,Category,ContextIndex,EliminatedBy,Feature,Generator,GeneratorFamilyIndex,GeneratorIndex,Sensors
0,Statistical,28,,gen_0029_AccelerometerYMedian,Median,1,9,[AccelerometerY]
1,Statistical,42,,gen_0043_AccelerometerXVariance,Variance,0,14,[AccelerometerX]
2,Statistical,44,,gen_0045_AccelerometerZVariance,Variance,2,14,[AccelerometerZ]
3,Statistical,49,,gen_0050_AccelerometerYmaximum,Maximum,1,16,[AccelerometerY]
4,Statistical,58,,gen_0059_AccelerometerY75Percentile,75th Percentile,1,19,[AccelerometerY]
5,Statistical,70,,gen_0071_AccelerometerYminimum,Minimum,1,23,[AccelerometerY]
6,Energy,73,,gen_0074_AvgDemeanedEng,Average Demeaned Energy,0,25,"[AccelerometerX, AccelerometerY, AccelerometerZ]"
7,Amplitude,79,,gen_0080_AccelerometerYMaxP2PGlobalAC,Global Peak to Peak of High Frequency,1,28,[AccelerometerY]
8,Amplitude,82,,gen_0083_AccelerometerYP2P,Global Peak to Peak,1,29,[AccelerometerY]
9,Amplitude,85,,gen_0086_AccelerometerYMinMaxSum,Global Min Max Sum,1,30,[AccelerometerY]


In [ ]:
dsk.pipeline.rehydrate(kp)